In [1]:
!pip install peft transformers evaluate accelerate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 12.8 MB/s eta 0:00:00


In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm
import pandas as pd
import evaluate
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Define the path to your Google Drive folder
drive_path = '/content/drive/MyDrive/Thesis/model_aya/'


In [3]:
peft_model_id = "/content/drive/MyDrive/Thesis/model_aya_checkpoints/checkpoint-5000"

In [4]:
config = PeftConfig.from_pretrained(peft_model_id)

In [5]:
config.base_model_name_or_path

'CohereForAI/aya-101'

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained(drive_path,  load_in_8bit=True,  device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(drive_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

model-00001-of-00011.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

model-00009-of-00011.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00010-of-00011.safetensors:   0%|          | 0.00/2.99G [00:00<?, ?B/s]

model-00011-of-00011.safetensors:   0%|          | 0.00/4.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [7]:
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(250112, 4096)
      (encoder): T5Stack(
        (embed_tokens): Embedding(250112, 4096)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear8bitLt(
                    (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=4096, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=4096, bias=False)
                    )
                    (lora_embedding_A): ParameterD

In [8]:
test_dataset = load_from_disk("/content/drive/MyDrive/Thesis/data/eval/").with_format("torch")

In [9]:
test_dataset

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
})

In [10]:
def evaluate_peft_model(sample, max_target_length=50):
    # generate definition using beam search
    outputs = model.generate(
        input_ids=sample["input_ids"].unsqueeze(0).cuda(),
        max_length=max_target_length,
        num_beams=4,
        early_stopping=True, 
        num_return_sequences=1
    )
    prediction = tokenizer.decode(outputs[0].cpu().numpy(), skip_special_tokens=True)
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)

    return prediction, labels

In [11]:
predictions, references = [] , []
for sample in tqdm(test_dataset):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append([l])

100%|██████████| 2000/2000 [3:25:59<00:00,  6.18s/it]


In [12]:
df = pd.DataFrame({'predictions': predictions, 'references': references})
df.to_csv('/content/drive/MyDrive/Thesis/humaneval/predictions_aya_full_test.csv', index=False)

In [13]:
output_dir = '/content/drive/MyDrive/Thesis/model_aya/'
model.base_model.save_pretrained(output_dir)


/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:399: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


In [17]:
metric = evaluate.load("rouge")

In [18]:
rouge = metric.compute(predictions=predictions, references=references)

In [19]:
rouge

{'rouge1': 0.27100479276598743,
 'rouge2': 0.09343865440899185,
 'rougeL': 0.2449715691296814,
 'rougeLsum': 0.24527883160818842}

In [20]:
smoother = SmoothingFunction().method4
bleu_score = corpus_bleu(references, predictions, smoothing_function=smoother, weights=(0.25, 0.25, 0.25, 0.25), auto_reweigh=True)
print(f"Adjusted BLEU score: {bleu_score * 100:.2f}%")

Adjusted BLEU score: 26.53%
